# Extração de Dados de Carcinoma em Exames de Biópsia de Mama do Fleury

## Introdução Técnica Detalhada

Este notebook implementa um pipeline para identificação e extração automatizada de resultados de carcinoma em laudos de biópsia de mama do laboratório Fleury. O sistema utiliza técnicas de processamento de linguagem natural (especificamente, expressões regulares) para detectar menções a carcinoma em relatórios de biópsia, focando na relação entre o termo "carcinoma" e a topografia "mama".

### Objetivo Principal

O objetivo principal deste notebook é processar laudos médicos de biópsia de mama para identificar aqueles que contêm diagnósticos de carcinoma. O sistema extrai esta informação crítica e a armazena de forma estruturada em uma tabela Delta para análises posteriores e integração com sistemas de monitoramento de saúde preventiva.

### Tecnologias Utilizadas

- **Framework de Processamento**: Apache Spark (PySpark)
- **Armazenamento de Dados**: Delta Lake
- **Processamento de Texto**: Expressões Regulares (RegEx)
- **Monitoramento e Alertas**: Octoops (Sentinel)
- **Logging**: Biblioteca padrão Python (logging)
- **Gerenciamento de Feature Store**: Databricks Feature Store

### Fluxo de Trabalho/Etapas Principais

1. **Configuração do Ambiente**: Instalação de dependências e configuração de logging
2. **Definição de Filtros**: Configuração de cláusulas SQL para filtragem incremental e por tipo de exame
3. **Extração de Carcinoma**: 
   - Consulta SQL com expressões regulares para identificar menções a carcinoma relacionadas à mama
   - Geração de indicador booleano `HAS_CARCINOMA`
4. **Seleção de Colunas**: Filtragem das colunas relevantes para análise e persistência
5. **Persistência dos Dados**: Salvamento dos dados processados em tabela Delta usando operações de merge
6. **Monitoramento**: Sistema de alertas para notificar em caso de ausência de dados ou erros

### Dados Envolvidos

- **Fonte de Dados**: 
  - Tabela: `refined.saude_preventiva.fleury_laudos`
  - Filtro principal: `linha_cuidado = 'mama'`
  - Tipos de exame específicos para biópsia: 'BIOMAMUSCORE', 'BIOMAMUSPAAF', 'BIOMAMUS', 'MAMOTOMIA', 'MAMOTOMUS', 'MAMCLIPE', 'MAMOTORM'

- **Tabela de Destino**:
  - `refined.saude_preventiva.fleury_laudos_mama_biopsia`: Tabela Delta contendo os dados extraídos de laudos de biópsia de mama com indicação de presença de carcinoma

- **Colunas Principais**:
  - `ficha`, `id_item`, `id_subitem`: Identificadores do exame
  - `laudo_tratado`: Texto do laudo médico processado
  - `RAW_CARCINOMA`: Extração bruta do termo encontrado
  - `HAS_CARCINOMA`: Indicador booleano da presença de carcinoma

### Resultados/Saídas Esperadas

1. **DataFrame Processado**: DataFrame contendo laudos de biópsia de mama com indicação de presença ou ausência de carcinoma
2. **Tabela Delta**: Persistência dos dados processados na tabela `refined.saude_preventiva.fleury_laudos_mama_biopsia`
3. **Contagem de Registros**: Exibição do número total de registros processados e salvos
4. **Alertas (se necessário)**: Notificações em caso de erros ou ausência de dados para processamento

### Pré-requisitos

- **Ambiente Databricks**: Com acesso ao Delta Lake e capacidade de execução de PySpark
- **Dependência Principal**: Pacote `octoops` para monitoramento e alertas
- **Acesso a Dados**: Permissões de leitura na tabela `refined.saude_preventiva.fleury_laudos`
- **Permissões de Escrita**: Para criar ou atualizar a tabela de destino

### Considerações Importantes

- **Processamento Incremental**: O notebook implementa um mecanismo de processamento incremental, processando apenas os registros mais recentes que os já salvos na tabela de destino
- **Expressões Regulares**: O padrão regex utilizado foi projetado especificamente para identificar a relação entre "carcinoma" e "mama" no texto dos laudos
- **Filtragem Demográfica**: O sistema foca em pacientes do sexo feminino na faixa etária de 40 a 75 anos
- **Tratamento de Erros**: Implementação robusta com captura de exceções e sistema de alertas via Sentinel
- **Processamento de Dados Sensíveis**: As informações processadas são sensíveis e relacionadas a diagnósticos de câncer, exigindo tratamento adequado de privacidade e segurança

# Extrair dados dos exames de Biopsia
+ Filtrar os laudos que possuem o termo topografia MAMA e desses laudos filtrar aqueles que possuem a palavra carcinoma (extrair toda a frase que está assoaciada ao termo).

## Instalação de Dependências

Esta célula instala a biblioteca `octoops`, que é necessária para o monitoramento e envio de alertas no pipeline de processamento. O Octoops é uma ferramenta interna utilizada para monitoramento de processos de dados, permitindo o envio de notificações e a gestão de feature stores.

A célula utiliza o comando mágico `%pip` do Jupyter para instalar a biblioteca diretamente no ambiente de execução. O código comentado mostra que originalmente também havia a intenção de instalar o pacote `databricks-feature-store` com a flag `-q` (quiet, para minimizar a saída do comando), mas esta instalação foi comentada.

Após a instalação desta biblioteca, será possível utilizar funcionalidades como:
- Sistema de alertas via Sentinel
- Gerenciamento de Feature Store
- Monitoramento de execução do pipeline

In [0]:
# %pip install databricks-feature-store -q
%pip install octoops

## Reinicialização do Ambiente Python

Esta célula reinicia o interpretador Python no ambiente Databricks através da função `dbutils.library.restartPython()`. Esta etapa é necessária após a instalação de novos pacotes (como o octoops instalado na célula anterior) para garantir que eles sejam carregados corretamente no ambiente de execução.

A reinicialização:
- Finaliza a sessão Python atual
- Inicia uma nova sessão com as bibliotecas recém-instaladas disponíveis
- Mantém o contexto do notebook (células já executadas)
- Garante que a biblioteca Octoops estará acessível para as células seguintes

Esta é uma prática recomendada ao instalar novas bibliotecas em notebooks Databricks, pois alguns pacotes só ficam disponíveis após a reinicialização do ambiente Python.

In [0]:
dbutils.library.restartPython()

## Importações Comentadas

Esta célula contém importações de bibliotecas que foram comentadas durante o desenvolvimento do notebook. Estas linhas representam uma abordagem alternativa ou anterior para configurar o ambiente Spark e o Feature Store, que não estão sendo utilizadas na versão atual do notebook.

As importações comentadas incluem:
- Módulos core do PySpark (`SparkSession`, `Window`, funções)
- Função `row_number` para potencial processamento de dados com numeração de linhas
- Inicialização personalizada de uma sessão Spark com nome específico ("LLM_Extractor_Carcinoma")
- Cliente do Databricks Feature Store

Estas importações foram mantidas como referência para possíveis modificações futuras ou para documentar abordagens alternativas que foram consideradas durante o desenvolvimento. Na implementação atual, as dependências necessárias são importadas na próxima célula de código.

In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.window import Window
# from pyspark.sql import functions as F
# from pyspark.sql.functions import row_number
# spark = SparkSession.builder.appName("LLM_Extractor_Carcinoma").getOrCreate()
# from databricks.feature_store import FeatureStoreClient

## Importação de Bibliotecas

Esta célula importa todas as bibliotecas e módulos necessários para o processamento dos dados, manipulação de DataFrames, operações com datas, logging e envio de alertas. As importações são essenciais para o funcionamento do pipeline de extração e processamento de dados de biópsia.

### Bibliotecas Importadas

#### PySpark SQL e Funções
- **Funções de coluna e manipulação de dados**: `col`, `year`, `month`, `dayofmonth`, `when`, `lit`, `expr`, `to_timestamp`
- **Tipos de dados**: `DateType` para manipulação de datas
- **DataFrame API**: `DataFrame` (para tipagem)
- **Funções de data**: `datediff`, `to_date` para cálculos com datas

#### Bibliotecas Padrão Python
- **Manipulação de datas**: `datetime`
- **Logging**: `logging` para registro de eventos e mensagens
- **Interação com sistema**: `sys`
- **Tratamento de erros**: `traceback` para formatação de mensagens de erro

#### Octoops (Monitoramento)
- `OctoOps`: Framework principal
- `Sentinel`: Componente para envio de alertas
- `FeatureStoreManager`: Gerenciador de feature store

#### Databricks
- `FeatureStoreClient`: Cliente para interagir com o Databricks Feature Store

Esta ampla gama de importações fornece as ferramentas necessárias para extrair, processar, validar e persistir os dados de laudos de biópsia de mama, bem como para monitorar a execução do pipeline e lidar com possíveis erros.

In [0]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel
from octoops import FeatureStoreManager
from databricks.feature_store import FeatureStoreClient

## Configuração do Logger

Esta célula configura um objeto logger para registrar mensagens de log durante a execução do notebook. O comentário incluído na célula original já explica sua função básica.

O logger é inicializado com a função `logging.getLogger(__name__)`, que cria um logger específico para o módulo atual (identificado por `__name__`). Esta abordagem é uma prática recomendada em Python para registrar eventos de forma organizada e hierárquica.

Benefícios desta configuração:
1. **Identificação da Fonte**: As mensagens de log são associadas ao módulo que as gerou
2. **Controle Granular**: Permite ajustar os níveis de logging (INFO, WARNING, ERROR) especificamente para este notebook
3. **Integração**: Funciona com o sistema de logging mais amplo do ambiente Databricks

Embora simples, esta configuração é essencial para o monitoramento eficiente do pipeline, permitindo que mensagens informativas e de erro sejam registradas de forma consistente ao longo da execução.

In [0]:
# O código configura um logger para registrar mensagens de log no módulo atual (__name__).
logger = logging.getLogger(__name__)

## Definição de Filtros para Extração de Dados

Esta célula configura os filtros e parâmetros para a extração dos dados de biópsia. Ela define três elementos principais: a tabela de destino e duas cláusulas SQL que serão utilizadas na consulta principal.

### Configurações Definidas

1. **Tabela de Destino**:
   ```python
   table_biopsia = "refined.saude_preventiva.fleury_laudos_mama_biopsia"
   ```
   Esta variável especifica o nome completo da tabela Delta onde os resultados serão salvos.

2. **Cláusula WHERE para Processamento Incremental** (`where_clause`):
   Esta cláusula implementa um mecanismo de processamento incremental, garantindo que apenas os registros mais recentes sejam processados. O filtro:
   - Compara o campo `_datestamp` dos registros na tabela fonte
   - Seleciona apenas registros com timestamp posterior ao último registro processado
   - Utiliza uma subconsulta para determinar o valor máximo de `_datestamp` na tabela de destino

3. **Filtro de Extração** (`filtro_extracao`):
   Este filtro define os critérios específicos para selecionar exames de biópsia de mama:
   - Linha de cuidado: 'mama'
   - Tipos específicos de exame: 'BIOMAMUSCORE', 'BIOMAMUSPAAF', 'BIOMAMUS', 'MAMOTOMIA', 'MAMOTOMUS', 'MAMCLIPE', 'MAMOTORM'
   - Sexo do paciente: feminino ('F')
   - Faixa etária: 40 a 75 anos

### Impacto no Pipeline

Estas configurações:
- Otimizam o processamento ao evitar reprocessamento desnecessário de dados
- Garantem que apenas exames relevantes de biópsia de mama sejam processados
- Aplicam filtros demográficos para focar na população-alvo
- Permitem a referência à tabela de destino em diferentes partes do código

In [0]:
# filtros para a extração
table_biopsia = "refined.saude_preventiva.fleury_laudos_mama_biopsia" 

where_clause = f"""
    WHERE
        _datestamp > (
            SELECT MAX(biop._datestamp)
            FROM {table_biopsia} biop
        )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado   = 'mama'
        AND sigla_exame IN (
            "BIOMAMUSCORE",
            "BIOMAMUSPAAF",
            "BIOMAMUS",
            "MAMOTOMIA",
            "MAMOTOMUS",
            "MAMCLIPE",
            "MAMOTORM"
        )
        AND UPPER(sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        )       
"""

## Consulta SQL para Extração de Carcinoma em Laudos de Biópsia

Esta célula contém o núcleo do processamento de dados, implementando uma consulta SQL complexa para identificar e extrair menções a carcinoma em laudos de biópsia de mama. Os comentários incluídos no código original já fornecem uma explicação básica sobre o funcionamento da expressão regular utilizada.

### Objetivo da Célula

A célula realiza três operações principais:
1. Define uma consulta SQL para extrair e processar laudos de biópsia de mama
2. Executa esta consulta utilizando o Spark SQL
3. Exibe o resultado em forma de tabela

### Estrutura da Consulta SQL

A consulta utiliza Common Table Expressions (CTEs) para organizar a lógica:

1. **CTE `base`**: 
   - Seleciona colunas da tabela `refined.saude_preventiva.fleury_laudos`
   - Calcula a idade do paciente a partir da data de nascimento
   - Aplica expressões regulares para extrair menções a carcinoma relacionadas à mama
   - Cria um campo booleano `HAS_CARCINOMA` que indica presença ou ausência de carcinoma

2. **Consulta Principal**: 
   - Seleciona todos os campos da CTE `base`
   - Aplica o filtro de extração definido anteriormente

### Expressões Regulares Utilizadas

O ponto central da consulta é o uso de expressões regulares para identificar carcinoma. Duas formas são identificadas:

```sql
REGEXP_EXTRACT(
    flr.laudo_tratado,
    r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
    1
)
```

Esta expressão:
1. Utiliza flags `(?mi)` para busca case-insensitive e multiline
2. Procura padrões onde:
   - "Topografia:" seguido de "MAMA" aparece antes de "CARCINOMA" OU
   - "CARCINOMA" aparece antes de "Topografia:" seguido de "MAMA"
3. Captura o termo "CARCINOMA" quando encontrado nestes contextos
4. Armazena o resultado em `RAW_CARCINOMA`

### Variáveis Criadas

- **`df_spk_biopsia`**: DataFrame Spark contendo os laudos de biópsia processados com indicação de presença de carcinoma

### Saída e Visualização

A função `display(df_spk_biopsia)` ao final exibe o DataFrame resultante, permitindo visualizar os laudos processados e verificar a detecção de carcinoma em cada um deles.

### Impacto

Esta consulta é o coração do notebook, realizando o processamento essencial para identificar diagnósticos de carcinoma em laudos de biópsia. O DataFrame criado será utilizado nas células posteriores para persistência e análise.

In [0]:
# Utiliza REGEXP_EXTRACT para buscar, no texto do campo 'laudo_tratado', 
# a ocorrência da palavra "CARCINOMA" relacionada à "MAMA" (seja antes ou depois de "Topografia").
# O padrão regex considera maiúsculas/minúsculas e múltiplas linhas.
# Se encontrar, extrai "CARCINOMA" para a coluna RAW_CARCINOMA; caso contrário, retorna string vazia.
# O CASE seguinte verifica se RAW_CARCINOMA está vazia:
# - Se estiver vazia, HAS_CARCINOMA recebe FALSE (não há carcinoma relacionado à mama no laudo).
# - Se não estiver vazia, HAS_CARCINOMA recebe TRUE (há carcinoma relacionado à mama no laudo).
query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        REGEXP_EXTRACT(
            flr.laudo_tratado,
            r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
            1
        ) AS RAW_CARCINOMA,
        CASE
            WHEN REGEXP_EXTRACT(
                    flr.laudo_tratado,
                    r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
                    1
                 ) = ''
            THEN FALSE
            ELSE TRUE
        END AS HAS_CARCINOMA
    FROM refined.saude_preventiva.fleury_laudos flr
    {where_clause}
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk_biopsia = spark.sql(query)
display(df_spk_biopsia)

## Visualização Adicional dos Dados Processados

Esta célula realiza uma visualização adicional do DataFrame `df_spk_biopsia` gerado na célula anterior. Esta segunda exibição pode parecer redundante (já que o DataFrame foi exibido ao final da célula anterior), mas ela serve a propósitos específicos:

1. **Inspeção Detalhada**: Permite visualizar o DataFrame de forma isolada, sem competir com a saída da execução da consulta SQL
2. **Verificação de Resultados**: Oferece uma oportunidade para examinar o conteúdo do DataFrame após possíveis manipulações intermediárias (caso existam)
3. **Depuração**: Facilita a verificação visual da estrutura e conteúdo dos dados antes de prosseguir para etapas de persistência

A função `display()` do Databricks apresenta os dados em formato tabular interativo, permitindo ordenação, filtragem e análise dos registros de forma mais conveniente do que métodos padrão como `df.show()`.

Esta visualização adicional é particularmente útil durante o desenvolvimento do notebook, permitindo aos analistas verificar visualmente o sucesso da detecção de carcinoma nos laudos de biópsia.

In [0]:
display(df_spk_biopsia)

## Configuração do Feature Store (Comentada)

Esta célula contém código comentado relacionado à criação de uma tabela no Databricks Feature Store. O código foi mantido como referência para implementação futura ou como documentação de uma abordagem alternativa que foi considerada durante o desenvolvimento.

### Funcionalidades Comentadas

O código comentado realiza as seguintes operações:

1. **Inicialização do Feature Store Client**:
   ```python
   fs = FeatureStoreClient()
   ```

2. **Criação de Tabela no Feature Store**:
   ```python
   fs.create_table(
       name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
       primary_keys=["ficha",'id_item','id_subitem'],
       schema=df_spk.schema,    
       description="Jornada de Mama - Features extraídas de laudos de biopsia Fleury. Siglas: BIOMAMUSCORE, BIOMAMUSPAAF, BIOMAMUS, MAMOTOMIA, MAMOTOMUS, MAMCLIPE, MAMOTORM")
   ```

### Configurações Importantes

O código comentado especifica:
- **Nome da tabela**: Idêntico ao definido na variável `table_biopsia`
- **Chaves primárias**: Combinação de `ficha`, `id_item` e `id_subitem` para identificação única de registros
- **Schema**: Baseado no DataFrame `df_spk` (que será definido na próxima célula)
- **Descrição**: Documentação detalhada sobre o propósito e o conteúdo da tabela

Na implementação atual, o notebook opta por usar diretamente as tabelas Delta para persistência dos dados, em vez de utilizar o Feature Store. Este código comentado é mantido para referência caso a abordagem de Feature Store seja adotada no futuro.

In [0]:
# FeatureStore
# fs = FeatureStoreClient()
# fs.create_table(
#     name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
#     primary_keys=["ficha",'id_item','id_subitem'],
#     schema=df_spk.schema,    
#     description="Jornada de Mama - Features extraídas de laudos de biopsia Fleury. Siglas: BIOMAMUSCORE, BIOMAMUSPAAF, BIOMAMUS, MAMOTOMIA, MAMOTOMUS, MAMCLIPE, MAMOTORM")

## Persistência de Dados e Tratamento de Erros

Esta célula final é responsável por selecionar, processar e persistir os dados extraídos na tabela Delta, além de implementar um robusto sistema de tratamento de erros. Esta é a etapa que efetivamente salva os resultados do processamento e garante que o sistema notifique os responsáveis em caso de falhas.

### Objetivo da Célula

Esta célula realiza várias operações críticas:
1. Importa bibliotecas adicionais para tratamento de erros e manipulação de tabelas Delta
2. Define constantes e caminhos para persistência de dados
3. Seleciona apenas as colunas relevantes do DataFrame
4. Define uma função para inserção de dados com operações de merge
5. Executa a persistência dos dados com tratamento de erros
6. Emite alertas em caso de ausência de dados ou erros

### Importações e Configurações

- **Importações adicionais**: Módulos `sys`, `traceback`, `Sentinel` e `DeltaTable`
- **Configuração de Webhook**: `WEBHOOK_DS_AI_BUSINESS_STG = 'stg'` para ambiente de homologação
- **Caminho de saída**: `OUTPUT_DATA_PATH = 'refined.saude_preventiva.fleury_laudos_mama_biopsia'`

### Seleção de Colunas

```python
df_spk = df_spk_biopsia.select("ficha","id_item","id_subitem","id_cliente","dth_pedido",
                           "dth_resultado", "sigla_exame","laudo_tratado","linha_cuidado",
                           "_datestamp","RAW_CARCINOMA","HAS_CARCINOMA")
```

Esta operação seleciona apenas as colunas necessárias para análise e persistência, eliminando colunas intermediárias ou desnecessárias.

### Função de Inserção de Dados

A função `insert_data()` implementa uma operação de merge (upsert) utilizando a API DeltaTable:
1. Carrega a tabela Delta existente
2. Configura uma operação de merge baseada na chave composta (`ficha`, `id_item`, `id_subitem`)
3. Atualiza registros existentes quando há correspondência
4. Insere novos registros quando não há correspondência

### Lógica de Tratamento de Erros

O bloco `try-except` implementa a seguinte lógica:
1. Verifica se o DataFrame contém registros (`df_spk.count() > 0`)
2. Se houver registros, persiste os dados utilizando a função `insert_data()`
3. Exibe a contagem de registros salvos
4. Se não houver registros, envia um alerta via Sentinel
5. Em caso de exceção em qualquer etapa, captura e relança o erro

### Sistema de Alertas

O sistema de alertas via Sentinel é configurado com:
- Nome do projeto: 'Monitor_Linhas_Cuidado_Mama'
- Ambiente: Definido pela constante `WEBHOOK_DS_AI_BUSINESS_STG`
- Título da tarefa: 'Fleury Mama Biopsia'
- Categoria do alerta: 'Alerta'
- ID descritivo: '2_fleury_mama_biopsia'

### Impacto

Esta célula finaliza o pipeline de processamento, garantindo que os dados extraídos sejam corretamente persistidos e que quaisquer problemas sejam prontamente notificados. O uso de operações de merge garante que os dados sejam atualizados de forma incremental, sem duplicações.

In [0]:
import sys
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'stg'


#OUTPUT_DATA_PATH = dbutils.widgets.get('OUTPUT_DATA_PATH')
OUTPUT_DATA_PATH = 'refined.saude_preventiva.fleury_laudos_mama_biopsia'

# Selecionar colunas de interesse
df_spk = df_spk_biopsia.select("id_marca", "id_unidade", "id_cliente", "id_ficha","ficha", "id_item","id_subitem","id_exame", "sigla_exame","dth_pedido","dth_resultado", "_datestamp","laudo_tratado","linha_cuidado","RAW_CARCINOMA","HAS_CARCINOMA")

# função para salvar dados na tabela
def insert_data(df_spk, output_data_path):  

    # Carrega a tabela Delta existente
    delta_table = DeltaTable.forName(spark, output_data_path)

    # Faz o merge (upsert)
    (delta_table.alias("target")
     .merge(
         df_spk.alias("source"),
         "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
     )
     .whenMatchedUpdateAll()
     .whenNotMatchedInsertAll()
     .execute())      
try:
    if df_spk.count() > 0:
        
        #1/0 
        insert_data(df_spk, OUTPUT_DATA_PATH)
        # fs.write_table(
        #     name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
        #     df=df_spk,
        #     mode="merge",
        # )
        print(f'Total de registros salvos na tabela: {df_spk.count()}')
            
    else:
        error_message = traceback.format_exc()
        summary_message = f"""Não há laudos para extração.\n{error_message}"""
        sentinela_ds_ai_business = Sentinel(
            project_name='Monitor_Linhas_Cuidado_Mama',
            env_type=WEBHOOK_DS_AI_BUSINESS_STG,
            task_title='Fleury Mama Biopsia'
        )

        sentinela_ds_ai_business.alerta_sentinela(
            categoria='Alerta', 
            mensagem=summary_message,
            job_id_descritivo='2_fleury_mama_biopsia'
        )
except Exception as e:
    traceback.print_exc()        
    raise e